# RCT-FACTS on EIS
This notebook will check, install and run [RCT tools](https://radical-cybertools.github.io/) and [FACTS](https://fact-sealevel.readthedocs.io/en/latest/index.html) locally on this machine.

In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime

Check if RCT tools are installed

In [ ]:
check_rct = !radical-stack
if 'radical.entk' in ' '.join(check_rct):
    print('RCT tools are installed')
else:
    rct_tools = ['saga', 'gtod', 'pilot', 'entk']
    for tool in rct_tools:
        print(f'Installing radical.{tool} is in progress')
        cmd_out = !pip install radical.{tool}
    print('Done')

!radical-stack

Check if `facts` repositry is present in the current path.

In [ ]:
if os.path.exists('facts'):
    print('FACTS repository already exist')
else:
    print('Clonning FACTS repository')
    out = !git clone -b facts_eis https://github.com/radical-collaboration/facts.git

Run FACTS dummy experiments using RCT tools.

In [ ]:
# do not use animated output in notebooks
os.environ['RADICAL_LOG_LVL'] ='DEBUG'
os.environ['RADICAL_REPORT_ANIME'] = 'False'
!cd facts && python3 runFACTS.py experiments/facts.demo --alt_id

# Plotting the Deconto16 Module results

In [ ]:
year = 2100 # Default year was 2150
module = 'GrIS'
module_set = 'FittedISMIP'
sheet = 'GIS'

# load fair temperation gsat data and projection data
exp_path = f'radical.pilot.sandbox/facts.None.facts.demo.{datetime.datetime.now().strftime("%m%d%Y").lower()}/pilot.0002'
fair_temp = f'{exp_path}/task.000002/facts.demo.temperature.fair.temperature_gsat.nc'
module_data = f'{exp_path}/task.000006/facts.demo.GrIS1f.FittedISMIP.GrIS_GIS_globalsl.nc'

# Open .nc data sets and restrict to specific year
x_585 = (xr.open_dataset(fair_temp).squeeze(drop = True).surface_temperature.sel(years = year, drop = True)).values
y_585 = (xr.open_dataset(module_data).squeeze(drop = True).sea_level_change.sel(years = year, drop = True)/10).values
quantile_spp585 = np.quantile(y_585,[.05,.17,.5,.83,.95])

plt.title(f'{module_set}/{module} - {sheet}')
plt.xlabel('FAIR GSAT')
plt.ylabel(f'{module_set.capitalize()}/{module} [cm]')

# Plot Data
plt.scatter(x_585,y_585,s=3,label=f'ssp585: {quantile_spp585}',color='blue')
plt.legend(loc="upper left")

The cell above displays the output generated by running a sample pipeline experiment using `Radical.entk` for the FACTS workflow. This pipeline consists of a single stage containing a single task named `preprocess.task1`.

When the task dependencies are resolved by EnTK, it is then handed over to the `Radical.pilot` runtime system. In this phase, the task goes through the following states: `SCHEDULED`, `SUBMITTED`, and eventually, `EXECUTED`. These transitions occur on the designated computing resources and in this case we used the `localhost` (i.e. the current virtual machine itself).

Once all of the tasks are executed, EnTK initiates the termination process for the allocated computing resources. As part of this finalization step, EnTK collects and compiles the necessary performance profiles and logs from the executed tasks.

Finally, it gracefully closes the session, ensuring that all resources are released, and the experiment is completed in an orderly manner.